<h2>0 Stage. Install needed packages and import necessary libraries</h2>

In [16]:
import requests
import pandas as pd
from pandas.io.html import read_html
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors


<h2>1 Stage. Download and link the needed tables</h2>

Create a list of pandas dataframes, scraping from the wikipedia page about Berlin boroughs.

In [17]:
page = "https://en.wikipedia.org/wiki/Boroughs_and_neighborhoods_of_Berlin"
wiki_tables = read_html(page, attrs={"class":"wikitable"}, header=0)

In [18]:
#Create initial table of boroughs for the aggregated analysis (2'nd part of this research)
boroughs_df = wiki_tables[0]
boroughs_df.drop(columns = ['Area in km²', 'Population 31 March 2010', 'Density per km²', 'Map'], inplace=True)
boroughs_df

,Borough
0,Charlottenburg-Wilmersdorf
1,Friedrichshain-Kreuzberg
2,Lichtenberg
3,Marzahn-Hellersdorf
4,Mitte
5,Neukölln
6,Pankow
7,Reinickendorf
8,Spandau
9,Steglitz-Zehlendorf


In [19]:
#Clean and concatenate localities table
borough_list = ['Mitte', 'Friedrichshain-Kreuzberg', 'Pankow', 'Charlottenburg-Wilmersdorf', 'Spandau', 'Steglitz-Zehlendorf', 'Tempelhof-Schöneberg', 'Neukölln', 'Treptow-Köpenick', 'Marzahn-Hellersdorf', 'Lichtenberg', 'Reinickendorf']
index = 0
for df, borough in zip(wiki_tables[1:], borough_list):
    df.drop(columns = ['Population as of 2008', 'Density inhabitants per km²', 'Map', 'Area in km²'], inplace=True)
    df['Locality'] = df['Locality'].map(lambda x: x[7:]) #remove digital code
    df['Borough'] = borough
localities_df = pd.concat(wiki_tables[1:], ignore_index=True)
localities_df.head()

,Locality,Borough
0,Mitte,Mitte
1,Moabit,Mitte
2,Hansaviertel,Mitte
3,Tiergarten,Mitte
4,Wedding,Mitte


Then let's finish preparation for localities analysis and add geo data.

In [20]:
latitudes = []
longitudes = []
localities = [] #for double check

for index, row in localities_df.iterrows():
    address = row['Locality'] + ', ' + row['Borough'] + ', Berlin, Germany'
    geolocator = Nominatim(user_agent='berlin')
    location = geolocator.geocode(address)
    latitudes.append(location.latitude)
    longitudes.append(location.longitude)
    localities.append(row['Locality'])
coord = pd.DataFrame({'Locality':localities ,'Latitude':latitudes, 'Longitude':longitudes})
coord.head()

,Locality,Latitude,Longitude
0,Mitte,52.517690,13.402376
1,Moabit,52.530102,13.342542
2,Hansaviertel,52.519123,13.341872
3,Tiergarten,52.509778,13.357260
4,Wedding,52.550123,13.341970


In [21]:
neighborhoods = pd.merge(localities_df, coord, on='Locality')
neighborhoods.head()

,Locality,Borough,Latitude,Longitude
0,Mitte,Mitte,52.517690,13.402376
1,Moabit,Mitte,52.530102,13.342542
2,Hansaviertel,Mitte,52.519123,13.341872
3,Tiergarten,Mitte,52.509778,13.357260
4,Wedding,Mitte,52.550123,13.341970


Our localities data is ready. We'll analyze popular venues in a radius of 1500 m, cause the square of most localities is less than 10sq.km. 

Let's prepare the table with borough information as well as download json data with coordinates of boroughs for visualization from <a href='https://github.com/m-hoerz/berlin-shapes'>here.</a>

We use payment for 1 sq.m. of flat (in euros) as a factor of borough price differentiation, cause there is no information about average rent price of boroughs. We use this <a href= 'https://www.firstcitiz.com/about-berlin/berlin-property-prices.html'>source</a> and transform data to csv file.

Boroughs quality index's been downloaded from <a href='https://berlin.placeilive.com/map'>here</a> and transformed to csv file. An index that is greater than 70 is good.

Migrants information's been downloaded from this <a href='https://en.wikipedia.org/wiki/Demographics_of_Berlin'>wiki page</a> and transformed to csv. We would like to live with the neighbors of our or close to us mentality,  so the priority is for the boroughs of russians, ukranians, etc. Mentality index - the higher index, the closer mentality. Note: it's a subject opinion of my family.

In [22]:
berlin_geo_json = r'berliner-bezirke.geojson' # geojson file

prices = pd.read_csv('property_price.csv')
prices.drop(columns = ['Unnamed: 0'], inplace=True)

quality = pd.read_csv('quality.csv')

migrants = pd.read_csv('mentality.csv')

boroughs = pd.merge(boroughs_df, prices, on='Borough')
boroughs = pd.merge(boroughs, quality, on='Borough')
boroughs = pd.merge(boroughs, migrants, on='Borough')
boroughs

,Borough,Property Price,Transportation,Daily life,Safety,Health,Sports,Entertainment,Demographics,Mentality index
0,Charlottenburg-Wilmersdorf,4100,91,80,60,73,88,94,72,0
1,Friedrichshain-Kreuzberg,4380,94,86,82,75,92,98,57,0
2,Lichtenberg,3480,89,76,54,69,90,75,55,2
3,Marzahn-Hellersdorf,2480,87,67,73,75,90,60,47,2
4,Mitte,4700,94,84,45,71,92,96,46,0
5,Neukölln,3300,90,77,70,76,84,82,40,0
6,Pankow,3870,89,72,44,70,82,79,82,1
7,Reinickendorf,2200,88,69,50,76,85,77,64,0
8,Spandau,3650,87,66,48,71,89,76,46,0
9,Steglitz-Zehlendorf,3500,89,73,64,77,75,90,82,1


<h2>2 Stage. Analysis of neighborhoods venues, using Foursquare API</h2>

In [23]:
#Define Berlin coordinates
address = 'Berlin, Germany'

geolocator = Nominatim(user_agent="skirka")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [50]:
# create map of Berlin using latitude and longitude values
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, locality in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Locality']):
    label = '{}, {}'.format(locality, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_berlin)  
    
map_berlin

In [51]:
#Now let's use Foursquare API data and transform it to the proper view.
CLIENT_ID = '5F4Z503LHA0TFNKY3IXOIXFHVYYMQP5L0DKJONG0NTNVZ0RW'
CLIENT_SECRET = '3W33JWTCF0L5GDTET3PZ1FP05JLE0T4HZPCPWZJEK022SVJZ'
VERSION = '20181226'
RADIUS = 1500
LIMIT = 100

In [52]:
#define function, which creates the dataframe of all neighborhoods and venues.
def getNearbyVenues(names, latitudes, longitudes):
    venues_list=[]

    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            RADIUS, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Locality', 
                  'Locality Latitude', 
                  'Locality Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [53]:
#Run the function
berlin_venues = getNearbyVenues(names=neighborhoods['Locality'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )
berlin_venues.head()

,Locality,Locality Latitude,Locality Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Mitte,52.51769,13.402376,Lustgarten,52.518469,13.399454,Garden
1,Mitte,52.51769,13.402376,Radisson Blu,52.519623,13.402488,Hotel
2,Mitte,52.51769,13.402376,Deutsches Historisches Museum,52.517788,13.396948,History Museum
3,Mitte,52.51769,13.402376,Altes Museum,52.519537,13.398803,History Museum
4,Mitte,52.51769,13.402376,Buchhandlung Walther König,52.521301,13.400758,Bookstore


Next step is to restructure the data for the analysis. The desired result is the following: localities should be in rows, different types of venues in columns, specific gravity of each type of venues as values.
We do it in 2 steps. First is to create dummies, next - group the rows as we need.

In [54]:
#one hot encoding
berlin_onehot = pd.get_dummies(berlin_venues[['Venue Category']], prefix="", prefix_sep="")

#add locality column back to dataframe
berlin_onehot['Locality'] = berlin_venues['Locality'] 

#move locality column to the first column
fixed_columns = [berlin_onehot.columns[-1]] + list(berlin_onehot.columns[:-1])
berlin_onehot = berlin_onehot[fixed_columns]

#group the rows by neighborhood
berlin_grouped = berlin_onehot.groupby('Locality').mean().reset_index()
berlin_grouped.head()

,Locality,ATM,Adult Boutique,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,...,Waterfront,Whisky Bar,Windmill,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Adlershof,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
1,Alt-Hohenschönhausen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,Alt-Treptow,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0
3,Altglienicke,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
4,Baumschulenweg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0


Now we have data for cluster analysis. To understand the reasons of clustering later, let's create one more dataframe, which shows top 10 venues for each locality. First, let's write a function to sort the venues in descending order. Then create the new dataframe and display the top 10 venues for each locality.

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [56]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to the number of top venues
columns = ['Locality']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
localities_venues_sorted = pd.DataFrame(columns=columns)
localities_venues_sorted['Locality'] = berlin_grouped['Locality']

for ind in np.arange(berlin_grouped.shape[0]):
    localities_venues_sorted.iloc[ind, 1:] = return_most_common_venues(berlin_grouped.iloc[ind, :], num_top_venues)

localities_venues_sorted.head()

,Locality,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adlershof,Supermarket,Café,Greek Restaurant,Hotel,Light Rail Station,Park,Credit Union,Coffee Shop,Gym / Fitness Center,Gym
1,Alt-Hohenschönhausen,Supermarket,Tram Station,Bakery,Lake,Drugstore,Go Kart Track,Bank,Bar,Gym / Fitness Center,Beach
2,Alt-Treptow,Café,Nightclub,Bar,Coffee Shop,Ice Cream Shop,Vietnamese Restaurant,Rock Club,German Restaurant,Pub,Beer Garden
3,Altglienicke,Supermarket,Light Rail Station,Intersection,Greek Restaurant,Bowling Alley,Bakery,Stadium,Electronics Store,Discount Store,Automotive Shop
4,Baumschulenweg,Supermarket,Café,Bus Stop,Drugstore,Indian Restaurant,Trattoria/Osteria,Forest,Pool,Smoke Shop,Garden Center


Now let's cluster our localities.

In [148]:
# set number of clusters
kclusters = 4

berlin_grouped_clustering = berlin_grouped.drop('Locality', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(berlin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 1, 0, 2, 0, 3, 2, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [149]:
berlin_merged = localities_venues_sorted

# add clustering labels
berlin_merged['Cluster'] = kmeans.labels_

# merge berlin_merged with neighborhoods to add latitude/longitude and other information for each locality
berlin_merged = berlin_merged.join(neighborhoods.set_index('Locality'), on='Locality')

# change the order of columns for better visualisation.
cols = berlin_merged.columns.tolist()

cols.insert(1, 'Borough')
cols.insert(2, 'Latitude')
cols.insert(3, 'Longitude')
cols.insert(4, 'Cluster')
cols = cols[:-4]

berlin_merged = berlin_merged[cols]

berlin_merged.head()

,Locality,Borough,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adlershof,Treptow-Köpenick,52.437893,13.547550,0,Supermarket,Café,Greek Restaurant,Hotel,Light Rail Station,Park,Credit Union,Coffee Shop,Gym / Fitness Center,Gym
1,Alt-Hohenschönhausen,Lichtenberg,52.549382,13.504673,2,Supermarket,Tram Station,Bakery,Lake,Drugstore,Go Kart Track,Bank,Bar,Gym / Fitness Center,Beach
2,Alt-Treptow,Treptow-Köpenick,52.492563,13.459874,0,Café,Nightclub,Bar,Coffee Shop,Ice Cream Shop,Vietnamese Restaurant,Rock Club,German Restaurant,Pub,Beer Garden
3,Altglienicke,Treptow-Köpenick,52.411838,13.542646,1,Supermarket,Light Rail Station,Intersection,Greek Restaurant,Bowling Alley,Bakery,Stadium,Electronics Store,Discount Store,Automotive Shop
4,Baumschulenweg,Treptow-Köpenick,52.461694,13.481548,0,Supermarket,Café,Bus Stop,Drugstore,Indian Restaurant,Trattoria/Osteria,Forest,Pool,Smoke Shop,Garden Center


Show results on the map

In [150]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(berlin_merged['Latitude'], berlin_merged['Longitude'], berlin_merged['Locality'], berlin_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now let's briefly analyse each cluster.

In [151]:
#The first cluster - mainly center of Berlin. The most popular places are cafes and restorants.
berlin_merged.loc[berlin_merged['Cluster'] == 0, berlin_merged.columns[[0] + list(range(5, berlin_merged.shape[1]))]]

,Locality,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adlershof,Supermarket,Café,Greek Restaurant,Hotel,Light Rail Station,Park,Credit Union,Coffee Shop,Gym / Fitness Center,Gym
2,Alt-Treptow,Café,Nightclub,Bar,Coffee Shop,Ice Cream Shop,Vietnamese Restaurant,Rock Club,German Restaurant,Pub,Beer Garden
4,Baumschulenweg,Supermarket,Café,Bus Stop,Drugstore,Indian Restaurant,Trattoria/Osteria,Forest,Pool,Smoke Shop,Garden Center
6,Blankenburg,Bakery,Bus Stop,Greek Restaurant,Supermarket,Gas Station,Restaurant,Café,Golf Course,Light Rail Station,German Restaurant
9,Borsigwalde,Supermarket,Metro Station,Drugstore,Italian Restaurant,Hotel,Eastern European Restaurant,Bakery,Motorcycle Shop,Shopping Mall,Trattoria/Osteria
13,Charlottenburg,Italian Restaurant,Gourmet Shop,Café,Chinese Restaurant,Coffee Shop,Bakery,Trattoria/Osteria,German Restaurant,Asian Restaurant,Korean Restaurant
14,Charlottenburg-Nord,Metro Station,Rental Car Location,Bus Stop,Fast Food Restaurant,Plaza,Park,Electronics Store,Sandwich Place,Beer Garden,Fair
15,Dahlem,Café,Supermarket,Museum,German Restaurant,Bakery,Asian Restaurant,Art Museum,Coffee Shop,Restaurant,Garden Center
18,Fennpfuhl,Supermarket,Italian Restaurant,Park,Vietnamese Restaurant,Hotel,Café,Drugstore,Tram Station,Pizza Place,Department Store
20,Friedenau,Café,Italian Restaurant,Trattoria/Osteria,Greek Restaurant,Ice Cream Shop,Park,Korean Restaurant,Bakery,Pub,German Restaurant


In [152]:
#The 2'nd cluster - periphery of Berlin. The most popular places are supermarkets and tram stations. 
berlin_merged.loc[berlin_merged['Cluster'] == 1, berlin_merged.columns[[0] + list(range(5, berlin_merged.shape[1]))]]

,Locality,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Altglienicke,Supermarket,Light Rail Station,Intersection,Greek Restaurant,Bowling Alley,Bakery,Stadium,Electronics Store,Discount Store,Automotive Shop
16,Falkenberg,Supermarket,Tram Station,Animal Shelter,Bus Stop,Coffee Shop,Zoo Exhibit,Fast Food Restaurant,Event Service,Event Space,Exhibit
34,Heiligensee,Supermarket,German Restaurant,Italian Restaurant,Soccer Field,Light Rail Station,Organic Grocery,Gas Station,Trattoria/Osteria,Lake,Zoo Exhibit
60,Neu-Hohenschönhausen,Supermarket,Tram Station,Light Rail Station,Shopping Mall,Electronics Store,Pet Café,Sports Bar,Drugstore,Movie Theater,Soccer Field
87,Wartenberg,Supermarket,Tram Station,Coffee Shop,Electronics Store,Movie Theater,Soccer Field,Bus Stop,Drugstore,Light Rail Station,Fast Food Restaurant


In [153]:
#The 3'd cluster - periphery of Berlin, but with good different places. There are supermarkets, as well as parks/lakes and cafes.
berlin_merged.loc[berlin_merged['Cluster'] == 2, berlin_merged.columns[[0] + list(range(5, berlin_merged.shape[1]))]]

,Locality,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Alt-Hohenschönhausen,Supermarket,Tram Station,Bakery,Lake,Drugstore,Go Kart Track,Bank,Bar,Gym / Fitness Center,Beach
5,Biesdorf,Supermarket,Light Rail Station,Furniture / Home Store,Italian Restaurant,Park,Palace,Greek Restaurant,Outdoor Event Space,Bakery,Trail
8,Bohnsdorf,Supermarket,Light Rail Station,Intersection,Discount Store,Greek Restaurant,Italian Restaurant,Drugstore,Hotel,Event Space,Exhibit
10,Britz,Supermarket,Bus Stop,Metro Station,Gas Station,Historic Site,Intersection,Bar,Snack Place,Chinese Restaurant,Park
11,Buch,Supermarket,Café,Health & Beauty Service,Lottery Retailer,Bus Stop,Gas Station,Light Rail Station,Art Gallery,Drugstore,Fast Food Restaurant
12,Buckow,Supermarket,Gas Station,Restaurant,Martial Arts Dojo,Soccer Field,Fast Food Restaurant,Greek Restaurant,Bus Stop,Italian Restaurant,Bakery
19,Französisch Buchholz,Supermarket,Tram Station,Greek Restaurant,Flower Shop,Hardware Store,Plaza,Drugstore,Organic Grocery,Argentinian Restaurant,Gas Station
21,Friedrichsfelde,Supermarket,Zoo Exhibit,Drugstore,Bakery,Bus Stop,Metro Station,Tram Station,Stadium,Event Space,Light Rail Station
27,Gropiusstadt,Supermarket,Metro Station,Hotel,Fast Food Restaurant,Bus Stop,Discount Store,Electronics Store,Chinese Restaurant,Gym,Gas Station
29,Grünau,Supermarket,Hotel,Drugstore,Liquor Store,Tram Station,Light Rail Station,Gastropub,Beach,River,Restaurant


In [154]:
#The 4'th cluster - periphery of Berlin. The most popular places are supermarket and bus stops. It's not bad zone with parks and lakes but without enough cafes and restaurants.
berlin_merged.loc[berlin_merged['Cluster'] == 3, berlin_merged.columns[[0] + list(range(5, berlin_merged.shape[1]))]]

,Locality,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Blankenfelde,Lake,Scenic Lookout,Park,Bus Stop,Zoo Exhibit,Event Service,Event Space,Exhibit,Fair,Falafel Restaurant
17,Falkenhagener Feld,Supermarket,Bus Stop,Drugstore,Bakery,Park,Trattoria/Osteria,Snack Place,Lake,Café,Farmers Market
25,Gatow,Bus Stop,Hotel,Beach,Harbor / Marina,Hardware Store,Supermarket,Trattoria/Osteria,Farmers Market,Scenic Lookout,German Restaurant
30,Hakenfelde,Supermarket,Bus Stop,Gas Station,Zoo Exhibit,Breakfast Spot,Drugstore,Hotel,Restaurant,Athletics & Sports,Turkish Restaurant
40,Karow,Supermarket,Plaza,Bus Stop,Lake,Boarding House,Bakery,BBQ Joint,Light Rail Station,German Restaurant,Doner Restaurant
41,Kaulsdorf,Supermarket,Hotel,Bus Stop,Market,Park,Drugstore,Light Rail Station,Shopping Mall,Diner,Lake
79,Staaken,Bus Stop,Supermarket,Drugstore,Fast Food Restaurant,Pet Store,Grocery Store,Castle,Bank,Automotive Shop,Train Station
92,Wilhelmstadt,Bus Stop,Supermarket,Pizza Place,Harbor / Marina,Lake,Pool,Restaurant,Fried Chicken Joint,Café,Gastropub


<h3>According to the cluster analysis the most comfortable clusters for my family are 0 and 2. There are a lot of supermarkets, cafes and places for leisure with children.</h3>

<h2>3 Stage. Analysis of boroughs</h2>

At this stage will do the following:
1. Chose the key factors for clusterization
2. Normalize data
3. Cluster boroughs on 3 groups, using the above-mentioned data
4. Rank all boroughs as 1-3 mark to show the preference for my family, taking to the consideration of all factors.

In [157]:
#Let's drop unnecessary columns

#We don't need borough for the cluster analysis
#Transportation and Health are not so differentiated
#Daily life, sports and entertainment are taken to the consideration in the locality analysis.
#Mentality index is a subject factor, so we take it to the consideration on the last step
df_agg_bor = boroughs.copy()
df_agg_bor.drop(columns=['Borough','Transportation','Health','Daily life','Sports','Entertainment', 'Mentality index'], inplace=True)

#Let's normalize data
for name in list(df_agg_bor):
    df_agg_bor[name] = df_agg_bor[name] / df_agg_bor[name].max()

df_agg_bor

,Property Price,Safety,Demographics,Cluster
0,0.872340,0.731707,0.878049,0.666667
1,0.931915,1.000000,0.695122,0.666667
2,0.740426,0.658537,0.670732,0.333333
3,0.527660,0.890244,0.573171,1.000000
4,1.000000,0.548780,0.560976,0.333333
5,0.702128,0.853659,0.487805,0.666667
6,0.823404,0.536585,1.000000,1.000000
7,0.468085,0.609756,0.780488,0.666667
8,0.776596,0.585366,0.560976,0.333333
9,0.744681,0.780488,1.000000,1.000000


In [158]:
# set number of clusters
kclusters_b = 3

# run k-means clustering
kmeans_b = KMeans(n_clusters=kclusters_b, random_state=0).fit(df_agg_bor)

# check cluster labels generated for each row in the dataframe
kmeans_b.labels_[0:10] 

array([0, 0, 2, 1, 2, 0, 1, 1, 2, 1], dtype=int32)

In [159]:
# add clustering labels
boroughs['Cluster'] = kmeans_b.labels_
boroughs

,Borough,Property Price,Transportation,Daily life,Safety,Health,Sports,Entertainment,Demographics,Mentality index,Cluster
0,Charlottenburg-Wilmersdorf,4100,91,80,60,73,88,94,72,0,0
1,Friedrichshain-Kreuzberg,4380,94,86,82,75,92,98,57,0,0
2,Lichtenberg,3480,89,76,54,69,90,75,55,2,2
3,Marzahn-Hellersdorf,2480,87,67,73,75,90,60,47,2,1
4,Mitte,4700,94,84,45,71,92,96,46,0,2
5,Neukölln,3300,90,77,70,76,84,82,40,0,0
6,Pankow,3870,89,72,44,70,82,79,82,1,1
7,Reinickendorf,2200,88,69,50,76,85,77,64,0,1
8,Spandau,3650,87,66,48,71,89,76,46,0,2
9,Steglitz-Zehlendorf,3500,89,73,64,77,75,90,82,1,1


In [160]:
#Good level of safety, but demographics level is average. We restructure this group by 2 in the next step
boroughs.loc[boroughs['Cluster'] == 0, boroughs.columns[:-1]]

,Borough,Property Price,Transportation,Daily life,Safety,Health,Sports,Entertainment,Demographics,Mentality index
0,Charlottenburg-Wilmersdorf,4100,91,80,60,73,88,94,72,0
1,Friedrichshain-Kreuzberg,4380,94,86,82,75,92,98,57,0
5,Neukölln,3300,90,77,70,76,84,82,40,0


In [161]:
#Average level of safety, but demographics level is enough high. We restructure this group by 2 in the next step
boroughs.loc[boroughs['Cluster'] == 1, boroughs.columns[:-1]]


,Borough,Property Price,Transportation,Daily life,Safety,Health,Sports,Entertainment,Demographics,Mentality index
3,Marzahn-Hellersdorf,2480,87,67,73,75,90,60,47,2
6,Pankow,3870,89,72,44,70,82,79,82,1
7,Reinickendorf,2200,88,69,50,76,85,77,64,0
9,Steglitz-Zehlendorf,3500,89,73,64,77,75,90,82,1
10,Tempelhof-Schöneberg,2750,90,76,67,75,81,86,73,1


In [162]:
# The last group isn't convenient for us, due to there is a low level of safety, property isn't cheap.
boroughs.loc[boroughs['Cluster'] == 2, boroughs.columns[:-1]]

,Borough,Property Price,Transportation,Daily life,Safety,Health,Sports,Entertainment,Demographics,Mentality index
2,Lichtenberg,3480,89,76,54,69,90,75,55,2
4,Mitte,4700,94,84,45,71,92,96,46,0
8,Spandau,3650,87,66,48,71,89,76,46,0
11,Treptow-Köpenick,3740,84,64,32,65,74,81,67,2


This clusterization doesn't fully meet my expectations. I've tried different columns set, number of clusters, different normalization methods, but the results were almost the same. So I've decided to create my own clusterization based on the above.

To the highest group (mark 3) I want to allocate indexes 3 (from the 1'st group), 6,9,10 (from the 2'nd group). It's safety group with average property prices.

Average group (mark 2) indexes 1,5 (from 1'st group), 0, 7 (from the 2'nd group). It's also a safe group, but either price is too high, or mentality index is zero.

Low group (mark 1) is the 3'd group without any changes. It's not a safety group with zero mentality index.

Let's restructure our analysis and look at the new clusters.This clusterization doesn't fully meet my expectations. I've tried different columns set, number of cluxters, different normalization methods, but the results were almost the same.
So I've decided to create my own clusterization based on the above.

To the highest group (mark 3) I want to allocate indexes 3 (from 1'st group), 6,9,10 (from the 2'nd group). It's safety group with average property prices.

Average group (mark 2) indexes 1,5 (from 1'st group), 0, 7 (from the 2'nd group). It's also safety group, but eather prices are too high, or mentality index is zero.

Low group (mark 1) is the 3'd group without any changes. It's not safety group with zero mentality index.

Let's restructure our analysis and look at the new clusters.

In [163]:
high = [3,6,9,10]
average = [1,5,0,7]
low = [2,4,8,11]

for ind in high:
    boroughs.at[ind,'Cluster'] = 3
for ind in average:
    boroughs.at[ind,'Cluster'] = 2
for ind in low:
    boroughs.at[ind,'Cluster'] = 1
    
boroughs.sort_values(by=['Cluster'])

,Borough,Property Price,Transportation,Daily life,Safety,Health,Sports,Entertainment,Demographics,Mentality index,Cluster
2,Lichtenberg,3480,89,76,54,69,90,75,55,2,1
4,Mitte,4700,94,84,45,71,92,96,46,0,1
8,Spandau,3650,87,66,48,71,89,76,46,0,1
11,Treptow-Köpenick,3740,84,64,32,65,74,81,67,2,1
0,Charlottenburg-Wilmersdorf,4100,91,80,60,73,88,94,72,0,2
1,Friedrichshain-Kreuzberg,4380,94,86,82,75,92,98,57,0,2
5,Neukölln,3300,90,77,70,76,84,82,40,0,2
7,Reinickendorf,2200,88,69,50,76,85,77,64,0,2
3,Marzahn-Hellersdorf,2480,87,67,73,75,90,60,47,2,3
6,Pankow,3870,89,72,44,70,82,79,82,1,3


<h2>4 Stage. Combine localities and boroughs analysis and show results on the map</h2>

First of all, let's add column for berlin_merged table with yellow color for prefferable localities and grey for not prefferable.

In [167]:
# define function with color
def color_df(df):
    if df['Cluster'] == 0 or df['Cluster'] == 2:
        return 'yellow'
    else:
        return 'grey'

berlin_merged_color = berlin_merged.copy()
berlin_merged_color['Color'] = berlin_merged_color.apply(color_df, axis=1)

At this stage we use Choropleth map to show the priority of boroughs and also mark our localities clusters.
It allows to choose the most appropriate localities.

In [168]:
# create map
map_final = folium.Map(location=[latitude, longitude], zoom_start=10)

# paint the boroughs 
map_final.choropleth(
    geo_data=berlin_geo_json,
    data=boroughs,
    columns=['Borough', 'Cluster'],
    key_on='feature.properties.spatial_alias',
    fill_color='BuGn', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Borough attractiveness'
)

# add markers to the map
for lat, lon, poi, cluster, clr in zip(berlin_merged_color['Latitude'], berlin_merged_color['Longitude'], berlin_merged_color['Locality'], berlin_merged_color['Cluster'], berlin_merged_color['Color']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color=clr,
        fill=True,
        fill_color=clr,
        fill_opacity=0.7).add_to(map_final)

# display map
map_final

<h3>As you can see, yellow circles on deep green background is prefferable localities for my family, the second priority is yellow circles on the light green background.</h3>
At the final stage let's create a table with preferable localities.

In [169]:
preffered_boroughs = ['Marzahn-Hellersdorf', 'Pankow', 'Steglitz-Zehlendorf', 'Tempelhof-Schöneberg']

localities_short = berlin_merged_color[berlin_merged_color['Borough'].isin(preffered_boroughs)]
localities_short = localities_short[localities_short['Color'] == 'yellow']

localities_short = pd.merge(localities_short, boroughs, on='Borough')

# Show only main factors for my family for easiest choice
columns_list = ['Borough', 'Locality', 'Property Price', 'Safety', 'Demographics', 'Mentality index', '1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', 'Latitude', 'Longitude']
localities_final = localities_short.copy()
localities_final = localities_final[columns_list]
localities_final.rename(columns = {'1st Most Common Venue':'1 Place', '2nd Most Common Venue':'2 Place', '3rd Most Common Venue':'3 Place', '4th Most Common Venue':'4 Place', '5th Most Common Venue':'5 Place'}, inplace=True)
localities_final.sort_values(by=['Safety'], ascending = False)

,Borough,Locality,Property Price,Safety,Demographics,Mentality index,1 Place,2 Place,3 Place,4 Place,5 Place,Latitude,Longitude
0,Marzahn-Hellersdorf,Biesdorf,2480,73,47,2,Supermarket,Light Rail Station,Furniture / Home Store,Italian Restaurant,Park,52.509001,13.555909
1,Marzahn-Hellersdorf,Hellersdorf,2480,73,47,2,Supermarket,Drugstore,Metro Station,Electronics Store,Bus Stop,52.536854,13.604774
2,Marzahn-Hellersdorf,Mahlsdorf,2480,73,47,2,Italian Restaurant,Supermarket,Fast Food Restaurant,Bakery,Gym / Fitness Center,52.506078,13.611859
3,Marzahn-Hellersdorf,Marzahn,2480,73,47,2,Garden,Tram Station,Supermarket,Shopping Mall,Café,52.542948,13.563142
26,Tempelhof-Schöneberg,Schöneberg,2750,67,73,1,Café,Italian Restaurant,Pizza Place,Cocktail Bar,French Restaurant,52.482157,13.355190
25,Tempelhof-Schöneberg,Marienfelde,2750,67,73,1,Supermarket,Bus Stop,Chinese Restaurant,Fast Food Restaurant,Recreation Center,52.412577,13.366592
24,Tempelhof-Schöneberg,Mariendorf,2750,67,73,1,Supermarket,Bakery,Bus Stop,Park,Fried Chicken Joint,52.440080,13.390028
23,Tempelhof-Schöneberg,Lichtenrade,2750,67,73,1,Supermarket,Electronics Store,Light Rail Station,Bakery,Doner Restaurant,52.393456,13.402040
22,Tempelhof-Schöneberg,Friedenau,2750,67,73,1,Café,Italian Restaurant,Trattoria/Osteria,Greek Restaurant,Ice Cream Shop,52.472075,13.329642
27,Tempelhof-Schöneberg,Tempelhof,2750,67,73,1,Supermarket,Café,Bus Stop,Bakery,Park,52.463292,13.386448
